In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split


D:\Tools\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


目前最好的参数：
ESP = 40
max_depth = 5


In [2]:
#seed = np.random.randint(99999)
seed = 71

np.random.seed(seed)
valid_size = 0.2
LOOP = 1
ESR = 40
# XGB param
nround = 1500
#nround = 10

param = {'max_depth':5, 
         'eta':0.05,
         'gamma ':0.1,
         'colsample_bytree':0.8,
         'subsample':0.8,
         'silent':1,
         'eval_metric':'auc',
         'objective':'binary:logistic',
        }

nofeatrue = ['PROJECT_END_FIRST_DIFF','BREAKFAITH_FIRST_RGYEAR_DIFF','BREAKFAITH_FIRST_END_FIRST_DIFF',
             'RIGHT_TYPE_RATE','RGYEAR_DIFF','PROJECT_INHOME_RATE','BREAKFAITH_FIRST_END_TIME',
             'PROJECT_INHOME_CNT','PROJECT_END_TIME','BREAKFAITH_END_FIRST_TIME','PROJECT_TYPECODE_CNT']



In [24]:
df_all = pd.read_csv('../data/alldata/df_data12.csv')

In [4]:
df_all.columns

Index(['EID', 'RGYEAR', 'HY', 'ZCZB', 'ETYPE', 'MPNUM', 'INUM', 'FINZB',
       'FSTINUM', 'TZINUM', 'HY_ZCZB_AVG', 'HY_ZCZB_AVG_RATE', 'MPNUM_HY_AVG',
       'INUM_HY_AVG', 'FINZB_HY_AVG', 'FSTINUM_HY_AVG', 'TZINUM_HY_AVG',
       'ETYPE_ZCZB_AVG', 'ETYPE_ZCZB_AVG_RATE', 'RGYEAR_DIFF', 'ZCZB_RATE',
       'ETYPE_RATE', 'MPNUM_HY_AVG_RATE', 'INUM_HY_AVG_RATE',
       'FINZB_HY_AVG_RATE', 'FSTINUM_HY_AVG_RATE', 'TZINUM_HY_AVG_RATE',
       'MPNUM_HY_RATE', 'INUM_HY_RATE', 'FINZB_HY_RATE', 'FSTINUM_HY_RATE',
       'TZINUM_HY_RATE', 'ALTERNO_SUM', 'ALTERNO_CNT', 'ALTERNO_CNT_RATE',
       'FIRST_CHANGE_TIME', 'END_CHANGE_TIME', 'IS_CHANGE',
       'CHANGE_FIRST_ALTERNO', 'CHANGE_END_ALTERNO', 'CHANGE_ALTERNO_MUCNID',
       'FIRST_CHANGE_TIME_DIFF', 'END_CHANGE_TIME_DIFF',
       'END_FIRST_CHANGE_TIME_DIFF', 'CHANGE_PRE_CNT', 'CHANGE_CNT_RATE',
       'CHANGE_HY_CNT_AVG', 'CHANGE_HY_CNT_ALL_AVG'],
      dtype='object')

In [25]:
timeType = ['RGYEAR','FIRST_CHANGE_TIME','END_CHANGE_TIME','BRANCH_FIRST_YEAR','BRANCH_END_YEAR',
            'BRANCH_FIRST_CLOSE_YEAR','TZ_QY_FIRST_TIME','TZ_QY_END_TIME','TZ_FIRST_CLOSE_TIME',
           'RIGHT_FIRST_ASK_TIME', 'RIGHT_FIRST_FB_TIME','RIGHT_END_ASK_TIME', 'RIGHT_END_FB_TIME',
           'PROJECT_FIRST_TIME', 'PROJECT_END_TIME', 'LAWSUIT_FIRST_TIME', 'LAWSUIT_END_TIME',            
            'BREAKFAITH_FIRST_FIRST_TIME', 'BREAKFAITH_FIRST_END_TIME','BREAKFAITH_END_FIRST_TIME',
           'RECRUIT_FIRST_TIME', 'RECRUIT_END_TIME']

timeType = ['RGYEAR','FIRST_CHANGE_TIME','END_CHANGE_TIME',]

def time2int(x):
    '''
    将时间转化为整数，加入训练
    2018-01 - x
    '''
    y = int(x[:x.find('-')])
    m = int(x[x.find('-')+1:])
    s = 12*(2018-y) + (1-m)
    
    return s

for t in timeType:
    df_all[t] = df_all[t].apply(time2int)



In [28]:

features = df_all.columns[1:]

def removeNOFeatrue(features,nofeatrue):
    features = list(features)
    for f in nofeatrue:
        features.remove(f)
    return features

# features = removeNOFeatrue(features,nofeatrue)

label = 'TARGET'

len(features)

93

In [29]:
# features = list(features)
# out = []

# for f in features:
#     if f.find('OUT')>=0:
#         out.append(f)
#         features.remove(f)

# len(features)    

In [30]:
def split_build_valid(df_train,k=5):
    '''
    k-fold交叉验证,默认k=10
    df_train:训练数据
    '''

    #added some parameters
    kf = KFold(n_splits = k, shuffle = True, random_state = 400000)    
    result1 = next(kf.split(df_train[df_train[label]==1]), None)
    result0 = next(kf.split(df_train[df_train[label]==0]), None)
    
#     print (result)
#     (array([0, 2, 3, 5, 6, 7, 8, 9]), array([1, 4]))
    train_list = []
    train_list.extend(result1[0])
    train_list.extend(result0[0])
    np.random.shuffle(train_list)
    
    vali_list = []
    vali_list.extend(result1[1])
    vali_list.extend(result0[1])
    np.random.shuffle(vali_list)
    
    
#     dtrain = df_train.iloc[train_list]
#     dvali =  df_train.iloc[vali_list]
    
    dbuild = xgb.DMatrix(df_train.iloc[train_list][features],label=df_train.iloc[train_list][label])
    dvalid = xgb.DMatrix(df_train.iloc[vali_list][features],label=df_train.iloc[vali_list][label])
    watchlist = [(dbuild, 'build'),(dvalid, 'valid')]
    
    return dbuild, dvalid, watchlist

def split_train_valid(df_train,test_size=0.2):
    '''
    k-fold交叉验证,默认k=10
    df_train:训练数据
    '''
    X_train, X_vali, y_train, y_vali = train_test_split(df_train[features], df_train[label], test_size=test_size, random_state=40000)
    #added some parameters
    
#     dtrain = df_train.iloc[train_list]
#     dvali =  df_train.iloc[vali_list]
    
    dtrain = xgb.DMatrix(X_train,label=y_train)
    dvalid = xgb.DMatrix(X_vali,label=y_vali)
    watchlist = [(dtrain, 'train'),(dvalid, 'valid')]
    
    return dtrain, dvalid, watchlist

In [31]:
def xtrain_and_test(df_all):
    '''
    得到训练数据和测试数据
    '''
    df_label = pd.read_csv('../data/public/train.csv')
    df_test_label = pd.read_csv('../data/public/evaluation_public.csv')
#     print(len(df_label))
#     print(len(df_test_label))
    df_train = df_all[df_all['EID'].isin(df_label['EID'])]
    df_test = df_all[df_all['EID'].isin(df_test_label['EID'])]
    
    df_train = pd.merge(df_train,df_label,how='left',on=['EID'])
    
    return df_train,df_test

df_train,df_test = xtrain_and_test(df_all)

In [32]:
print(len(df_train[df_train[label] == 1][label]))
print(len(df_train[df_train[label] == 0][label]))

29092
123914


In [33]:
models = []
for i in range(LOOP):
    print('LOOP',i)
    dbuild, dvalid, watchlist = split_train_valid(df_train,test_size=0.2)
            
    model = xgb.train(param, dbuild, nround, watchlist,early_stopping_rounds=ESR,verbose_eval=5)
    models.append(model)
    model.save_model('../saveModel/model1234'+ str(i) + '.model')
    # VALID
    valid_yhat = model.predict(dvalid,ntree_limit=model.best_iteration)
    print('Valid Mean:---------------------->', np.mean(valid_yhat))
    del dbuild, dvalid, watchlist

#==============================================================================
print('test')
#==============================================================================

dtest  = xgb.DMatrix(df_test[features])
proba_test = pd.DataFrame()
proba_test['EID'] = df_test['EID']
proba_test['FORTARGET'] = [0 for i in range(len(df_test))]
proba_test['PROB'] = [0 for i in range(len(df_test))]
for model in models:
    proba_test['PROB'] += model.predict(dtest)
proba_test['PROB'] /= LOOP




LOOP 0
[0]	train-auc:0.656928	valid-auc:0.64586
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[5]	train-auc:0.669052	valid-auc:0.655491
[10]	train-auc:0.673042	valid-auc:0.660617
[15]	train-auc:0.675565	valid-auc:0.662708
[20]	train-auc:0.677344	valid-auc:0.663971
[25]	train-auc:0.679229	valid-auc:0.665516
[30]	train-auc:0.681007	valid-auc:0.667065
[35]	train-auc:0.683069	valid-auc:0.668343
[40]	train-auc:0.684947	valid-auc:0.669765
[45]	train-auc:0.686442	valid-auc:0.670656
[50]	train-auc:0.687919	valid-auc:0.671605
[55]	train-auc:0.689281	valid-auc:0.672426
[60]	train-auc:0.69058	valid-auc:0.673062
[65]	train-auc:0.691717	valid-auc:0.673558
[70]	train-auc:0.693192	valid-auc:0.674288
[75]	train-auc:0.694432	valid-auc:0.674869
[80]	train-auc:0.69537	valid-auc:0.674958
[85]	train-auc:0.696433	valid-auc:0.675431
[90]	train-auc:0.697534	valid-auc:0.675822
[95]	train-auc:0.698577	valid-auc:0.67

In [34]:
import operator
importance = model.get_fscore()
importance = sorted(importance.items(), key=operator.itemgetter(1),reverse=True)
importance

[('CHANGE_PRE_CNT', 678),
 ('ZCZB_RATE', 485),
 ('ZCZB_RATE_ETYPE_RATE', 484),
 ('CHANGE_PRE_CNT_ZCZB_RATE', 434),
 ('FSTINUM_HY_RATE_ZCZB', 416),
 ('INUM_HY_RATE', 363),
 ('MPNUM_HY_RATE_FIRST_CHANGE_TIME_DIFF', 353),
 ('MPNUM_HY_RATE', 349),
 ('ETYPE_RATE', 345),
 ('FSTINUM_HY_RATE', 326),
 ('END_CHANGE_TIME_MPNUM', 319),
 ('INUM_ETYPE_ZCZB_AVG', 294),
 ('FIRST_CHANGE_TIME_DIFF', 290),
 ('ZCZB', 288),
 ('ALTERNO_CNT_FSTINUM', 287),
 ('FIRST_CHANGE_TIME_CHANGE_END_ALTERNO', 287),
 ('FIRST_CHANGE_TIME_DIFF_END_CHANGE_TIME_DIFF', 284),
 ('INUM_HY_RATE_FSTINUM_HY_RATE', 273),
 ('MPNUM_HY_AVG_RGYEAR', 266),
 ('ALTERNO_CNT', 260),
 ('ETYPE_RATE_INUM_HY_RATE', 256),
 ('END_CHANGE_TIME_DIFF_ALTERNO_CNT', 248),
 ('END_CHANGE_TIME_DIFF', 247),
 ('CHANGE_ALTERNO_MUCNID_INUM', 244),
 ('END_CHANGE_TIME', 243),
 ('FSTINUM_HY_AVG_ETYPE', 232),
 ('INUM_HY_AVG_CHANGE_FIRST_ALTERNO', 220),
 ('FIRST_CHANGE_TIME', 216),
 ('MPNUM_ALTERNO_SUM', 207),
 ('FSTINUM_END_CHANGE_TIME', 202),
 ('ALTERNO_SUM', 198

In [14]:
xfeature = [f for (f,v) in importance]



In [41]:
proba_test.loc[proba_test['PROB']>=0.45,'FORTARGET'] = 1

In [42]:
proba_test.to_csv('../result1/submussion_123456789_ESP40_5__.csv',index=False,index_label=False)

In [43]:
print(len(proba_test[proba_test['FORTARGET']==1]))
print(len(proba_test[proba_test['FORTARGET']==0]))


3902
98222


In [18]:
# df_label = pd.read_csv('../data/public/train.csv')

In [19]:
print(len(df_label[df_label['TARGET']==1]))
print(len(df_label[df_label['TARGET']==0]))

29092
123914
